In [ ]:
#bug: maturity and volume in the header
#bug: product_type column is eliminated

In [14]:
%load_ext autoreload
%autoreload 2
import pickle
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from zlu_functions import del_subset_col, set_col_headers, \
                          row_header_to_col,remove_rows_for_total_value, \
                          dfs_need_transpose, remove_cols_for_fair_value, \
                          collar_fill_empty,remove_empty_rows_and_cols, \
                          df_column_uniquify, verticalSplit_instrument, \
                          verticalSplit_ZLU
from nel_functions import delete_uninformative,delete_empty,delete_duplicated, \
                          delete_subset,get_vertically_stacked_subtables, \
                          vertically_stacked_subtable_cleaner
from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary,verticalSplit, \
                   instrumentCleaner, productCleaner, filterBusiness,columnSorter, \
                   volumeCleaner_specialSauce
#pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')

pickle_in=open('hedge_dfs_subset_8_18.pickle', 'rb')
df_dic=pickle.load(pickle_in)

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_1 = delete_subset(df_dic)

ticker_list = []

for k in df_dic.keys():
    if k[:3] not in ticker_list:
        print(k[:3])
        ticker_list.append(k[:3])


df_dic = filterBusiness(df_dic, 'CHK')
#ZLU change
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        df_col="".join([x.lower() for x in df.columns])
        if 'december' not in df_col and len(df_dic[k])>1: df_dic[k][i]=pd.DataFrame()
        else: print(k,i)
        
delete_empty(df_dic)

#ZlU change


       
#ZlU change
# df_dic={'CHK20100301':df_dic['CHK20100301']}
#del df_dic['PKD20090302'] #false positive
with open('output_before_spliting.csv','w') as f:
    pass
#preprocessing for all dfs
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        
        df=df_dic[k][i]
        df=set_col_headers(df) #set headers
        df=del_subset_col(df) #delete redundant cols
        df=remove_rows_for_total_value(df) #if any row for total value; save it
        df=remove_cols_for_fair_value(df)
        df_dic[k][i]=df
        with open('output_before_spliting.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

        print(k,'Original Table:',i,'-'*10)
        display(HTML(df.to_html()))

#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    #print(k)
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    elif k=='BRY20110301':
        split_dfs = []
        for table in df_dic[k]:
            df1 = table.iloc[:, 0:3]
            df2 = table.iloc[:, 3:]
            split_dfs.append(df1)
            split_dfs.append(df2)
            df_dic[k]=split_dfs
    else: df_dic[k] = verticalSplit(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
 

for k in df_dic.keys():
    #print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=row_header_to_col(df)
            df=dfs_need_transpose(df)
            df=collar_fill_empty(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
#             print(k,'Table:',i,'-'*10,'After preprocessing')
#             display(HTML(df.to_html()))

#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    print(k)
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])    
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
           
#Horizontally Split
del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_3 = delete_subset(df_dic)

for k in df_dic.keys(): #apply Sam's spliting function
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
            
with open('output_before_sam.csv','w') as f:
    pass
with open('output_after_sam.csv','w') as f:
    pass
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('output_before_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

date_keys = []               
date_stop = []

price_keys = ['weighted average strikeprice','fixedprice to bereceived','weighted average floor price per mcf','weighted average price per mcf',
              'weighted average price per bbl', 'weighted average floor price per bbl',
              'weighted,average,price to be,swapped,($/mmbtu),,', 'weighted,average,floor,price,($/mmbtu),,',
              'weighted,average,ceiling,price,($/mmbtu),,', 'weighted,average,basis,differential,($/mmbtu),,',
              'weighted average price per mmbtu', 'weighted average price', 'weighted average floor price',
              'weighted average ceiling price', 'weighted average basis differential','call','put','weightedaveragedifferential']
price_stop = []

volume_keys = ['(bcf)', '(mbbls)', 'volume']
volume_stop = ['natural gas (bcf)']               

#set verbose to True to turn on the diagonstic output for Sam's functions
verbose = False
key_override = True
                
for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
#         display(HTML(df.to_html()))
        if df.shape[0]>0: #apply Sam's functions to non-empty dfs
            print(k,'Table:',i,'-'*10,'Before Sam function')
            display(HTML(df.to_html()))
            df = instrumentCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = productCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = priceCleaner(df, key_words = price_keys, stop_words = price_stop,\
                                  verbose = verbose, key_override = key_override)
            df = volumeCleaner(df, key_words = volume_keys, stop_words = volume_stop,\
                                   verbose = verbose, key_override = False)
            df = dateCleaner(df, key_words = date_keys, stop_words = date_stop,\
                                 verbose = verbose, new_override = True, value_ids = ["20\d{2}"])
            
            
#                     #ZLU's change begins
#             col_lst=[col for col in df.columns if 'vol' in col]
#             for col in col_lst:
#                 df.loc[:,col]=df.loc[:,col].str.replace('[,\(\)]',"")
#     #         display(HTML(df.to_html()))
#             col_lst=['vol' in col for col in df.columns]
#             row_lst=list(range(df.shape[0]))
#             for row_idx in range(df.shape[0]):
#                 if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
#                     row_lst.remove(row_idx)
#             df=df.iloc[row_lst]
#             if all(df.loc[:,'vol_1_type']=='volume'): df.loc[:,'vol_1_type']=df.loc[:,'product_type'].str.extract('\(([^\)]+)\)')
#             #ZLU's change ends
            
            
            print(k+' ,table:'+str(i)+'-----After Sam functions----')
            display(HTML(df.to_html()))
            print('~'*50)
        else: df=pd.DataFrame()
        df_dic[k][i]=df

del_dfs_dic_5 = delete_uninformative(df_dic)
print('1',len(df_dic))
delete_empty(df_dic)
print('2',len(df_dic))
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
print('3',len(df_dic))
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
print('4',len(df_dic))
del_dfs_dic_3 = delete_subset(df_dic)
print('5',len(df_dic))

print(len(df_dic.keys()))

big_df = pd.DataFrame()

for k in df_dic.keys():
    doc_id = k
    processed_dfs = df_dic[k]
    for df in processed_dfs:
        unprocessed_cols = columnSorter(df)
        print(unprocessed_cols)
#         df.drop(unprocessed_cols, axis = 1, inplace = True)
    
        
        
#         display(HTML(df.to_html()))
        length = df.shape[0]
        doc_id_col = np.array([doc_id]*length)
        df.insert(0, 'doc_id', doc_id_col)
        big_df = pd.concat([big_df, df], axis = 0, join = 'outer', sort = False)

print(big_df.shape)


try: big_df.to_csv('CHK_clean.csv', encoding='cp1252')
except: big_df.to_csv('CHK_clean.csv', encoding = 'utf-8')
# for k in df_dic.keys():
#     for i in range(len(df_dic[k])):
#         with open('APA_clean.csv','a') as f:
#             df=df_dic[k][i]
#             f.writelines(k+'\n'+'table:'+str(i))
#             try: df.to_csv(f,encoding='cp1252')
#             except: df.to_csv(f,encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
AHC
APA
APC
ARE
CHK
CLR
COG
COP
CPE
CRC
CRK
CRZ
CXO
DNR
DVN
ECA
EOG
GPO
HES
LPI
MRO
MTD
MUR
NBL
NOG
OAS
OXY
PDC
PE2
PET
PXD
QEP
RRC
SM2
SWN
UNT
WLL
WPX
WTI
XEC
CHK20030227 0
CHK20040315 0
CHK20050309 0
CHK20060314 0
CHK20060314 1
CHK20060314 2
CHK20070301 0
CHK20070301 1
CHK20070301 2
CHK20080229 0
CHK20080229 1
CHK20080229 2
CHK20090302 0
CHK20090302 1
CHK20100301 2
CHK20110301 2
CHK20120229 2
CHK20130301 2
CHK20140227 2
CHK20150227 2
CHK20160225 1
CHK20030227 Original Table: 0 ----------


,,Volume,Average Strike Price,Weighted- Average Put Strike Price,Weighted- Average Differential to Mid- Continent Points,SFAS 133 Hedge
0,natural gas (mmbtu),,,,,
1,Swaps,,,,,
2,2003,57150000,4.31,,,Yes
3,Cap-Swaps,,,,,
4,2003,51100000,3.60,2.60,,No
5,Basis Protection Swaps,,,,,
6,2003,91250000,,,Neg0.15,No
7,2004,91500000,,,Neg0.15,No
8,2005,98550000,,,Neg0.16,No
9,2006,36500000,,,Neg0.16,No


CHK20040315 Original Table: 0 ----------


,,Volume,Weighted Average StrikePrice,WeightedAveragePutStrikePrice,Weighted Average Call StrikePrice,Weighted Average Differential,SFAS 133Hedge,PremiumsReceived
1,Natural Gas (mmbtu),,,,,,,
2,Swaps,,,,,,,
3,2004,171120000,5.31,,,,Yes,
4,2005,45550000,4.82,,,,Yes,
5,2006,25550000,4.74,,,,Yes,
6,2007,25550000,4.76,,,,Yes,
8,Basis ProtectionSwaps,,,,,,,
9,2004,157380000,,,,Neg0.17,No,
10,2005,109500000,,,,Neg0.16,No,
11,2006,47450000,,,,Neg0.16,No,


CHK20050309 Original Table: 0 ----------


...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CHK20160225
CHK20160225 Table: 0 ---------- Before Sam function


,",,,",",december 31, 2015,volume,",",december 31, 2014,volume,",product_type,instrument_type
6,Three-way collars,13.5,4.4,oil (mmbbl),
7,Call options,19.2,35.8,oil (mmbbl),
13,Three-way collars,,207,natural gas (tbtu),basis protection swaps
14,Call options,295,193,natural gas (tbtu),basis protection swaps
15,Basis protection swaps,57,60,natural gas (tbtu),basis protection swaps


instrument_type column already exists
CHK20160225 ,table:0-----After Sam functions----


,",,,",vol_1,vol_1_type,vol_2,vol_2_type,instrument_type
6,Three-way collars,13.5,",december 31, 2015,volume,",4.4,",december 31, 2014,volume,, oil (mmbbl)",three-way collar
7,Call options,19.2,",december 31, 2015,volume,",35.8,",december 31, 2014,volume,, oil (mmbbl)",call
13,Three-way collars,,",december 31, 2015,volume,",207,",december 31, 2014,volume,, natural gas (tbtu)",basis protection swaps three-way collar
14,Call options,295,",december 31, 2015,volume,",193,",december 31, 2014,volume,, natural gas (tbtu)",basis protection swaps call
15,Basis protection swaps,57,",december 31, 2015,volume,",60,",december 31, 2014,volume,, natural gas (tbtu)",basis protection swaps


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 14
2 14
3 14
4 14
5 14
14
['average strike price', 'sfas\xa0133 hedge']
['sfas 133hedge', 'premiumsreceived']
['sfas 133hedge', 'premiumsreceived']
['sfas 133hedge', 'net premiumsreceived ($ inthousands)']
['sfas 133hedge,']
['sfas 133hedge']
['sfas 133 hedge']
['sfas 133 hedge,', 'net premiums received ($ in thousands),']
['sfas 133 hedge']
['sfas 133hedge', 'netpremiums($ inmillions)']
['sfas 133hedge,', 'netpremiums($ inmillions),']
['sfas 133hedge']
['sfas 133hedge', 'netpremiums($ in millions)']
['sfas 133hedge', 'netpremiums($ in millions)']
[',,']
[',,']
[',,']
[',,,']
[',,,']
[',,,']
[',,,']
(426, 30)


In [ ]:
big_df['fake column'] is not None